## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Data Extraction

In [ ]:
dataset = pd.read_csv("/content/Airline_review.csv")

In [ ]:
dataset

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,23166,ZIPAIR,1,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,Couple Leisure,Economy Class,Bangkok to Tokyo,June 2022,2.0,1.0,NaN,1.0,1.0,1.0,1.0,no
23167,23167,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,Solo Leisure,Economy Class,Singapore to Tokyo,June 2022,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no
23168,23168,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,Business,Economy Class,Bangkok to Tokyo,May 2022,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no
23169,23169,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,Dreamliner,Business,Business Class,Tokyo to Los Angeles,May 2022,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes


In [ ]:
dataset.columns

Index(['Unnamed: 0', 'Airline Name', 'Overall_Rating', 'Review_Title',
       'Review Date', 'Verified', 'Review', 'Aircraft', 'Type Of Traveller',
       'Seat Type', 'Route', 'Date Flown', 'Seat Comfort',
       'Cabin Staff Service', 'Food & Beverages', 'Ground Service',
       'Inflight Entertainment', 'Wifi & Connectivity', 'Value For Money',
       'Recommended'],
      dtype='object')

In [ ]:
dataset = dataset[dataset['Verified'] != False]

In [ ]:
dataset

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no
5,5,Adria Airways,1,"""Had very bad experience""",17th September 2019,True,Had very bad experience with rerouted and ca...,CR 900,Couple Leisure,Economy Class,Sarajevo to Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23164,23164,ZIPAIR,1,"""$300 savings was absolutely not worth it""",22nd July 2022,True,My issues started before we even flew. I rec...,NaN,Family Leisure,Economy Class,Los Angeles to Narita,July 2022,3.0,4.0,1.0,1.0,1.0,2.0,2.0,no
23167,23167,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,Solo Leisure,Economy Class,Singapore to Tokyo,June 2022,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no
23168,23168,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,Business,Economy Class,Bangkok to Tokyo,May 2022,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no
23169,23169,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,Dreamliner,Business,Business Class,Tokyo to Los Angeles,May 2022,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes


In [ ]:
features=['Review_Title','Review','Recommended']
dataset=dataset[features]

In [ ]:
#checking null values
dataset.isnull().sum()

Review_Title    0
Review          0
Recommended     0
dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

dataset['Recommended'] = label_encoder.fit_transform(dataset['Recommended'])

<ipython-input-94-2591c3eb7922>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Recommended'] = label_encoder.fit_transform(dataset['Recommended'])


##Explanation

In [ ]:
dataset['Review_Title'] = dataset['Review_Title'].str.replace('"', '')

<ipython-input-95-b2ae88507565>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Review_Title'] = dataset['Review_Title'].str.replace('"', '')


In [ ]:
dataset['final_review']=dataset['Review_Title']+dataset['Review']

<ipython-input-96-f9f59eeb3cbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['final_review']=dataset['Review_Title']+dataset['Review']


In [ ]:
dataset['final_review']

0        pretty decent airline  Moroni to Moheli. Turne...
1        Not a good airline Moroni to Anjouan. It is a ...
2        flight was fortunately short  Anjouan to Dzaou...
4        it ruined our last days of holidays Do not boo...
5        Had very bad experience  Had very bad experien...
                               ...                        
23164    $300 savings was absolutely not worth it  My i...
23167    Avoid at all costs  Avoid at all costs. I book...
23168    Will not recommend to anyone  Flight was leavi...
23169    It was immaculately clean Zipair is JAL’s budg...
23170    lost all of our money with no refund  They lie...
Name: final_review, Length: 12322, dtype: object

In [ ]:
review=dataset['final_review'][1]

In [ ]:
#a) remove unneccessary .,
review=re.sub('[^a-zA-Z]',' ',review)
review

'Not a good airline Moroni to Anjouan  It is a very small airline  My ticket advised me to turn up at     hrs which I did  There was confusion at this small airport  I was then directed to the office of AB Aviation which was still closed  It opened at     hrs and I was told that the flight had been put back to     hrs and that they had tried to contact me  This could not be true as they did not have my phone number  I was with a local guide and he had not been informed either  I presume that I was bumped off  The later flight did operate but as usual  there was confusion at check in  The flight was only   mins and there were no further problems  Not a good airline but it is the only one for Comoros '

In [ ]:
#b) lower case the text
review=review.lower()
review

'not a good airline moroni to anjouan  it is a very small airline  my ticket advised me to turn up at     hrs which i did  there was confusion at this small airport  i was then directed to the office of ab aviation which was still closed  it opened at     hrs and i was told that the flight had been put back to     hrs and that they had tried to contact me  this could not be true as they did not have my phone number  i was with a local guide and he had not been informed either  i presume that i was bumped off  the later flight did operate but as usual  there was confusion at check in  the flight was only   mins and there were no further problems  not a good airline but it is the only one for comoros '

In [ ]:
#c) split the text
review=review.split()
review

['not',
 'a',
 'good',
 'airline',
 'moroni',
 'to',
 'anjouan',
 'it',
 'is',
 'a',
 'very',
 'small',
 'airline',
 'my',
 'ticket',
 'advised',
 'me',
 'to',
 'turn',
 'up',
 'at',
 'hrs',
 'which',
 'i',
 'did',
 'there',
 'was',
 'confusion',
 'at',
 'this',
 'small',
 'airport',
 'i',
 'was',
 'then',
 'directed',
 'to',
 'the',
 'office',
 'of',
 'ab',
 'aviation',
 'which',
 'was',
 'still',
 'closed',
 'it',
 'opened',
 'at',
 'hrs',
 'and',
 'i',
 'was',
 'told',
 'that',
 'the',
 'flight',
 'had',
 'been',
 'put',
 'back',
 'to',
 'hrs',
 'and',
 'that',
 'they',
 'had',
 'tried',
 'to',
 'contact',
 'me',
 'this',
 'could',
 'not',
 'be',
 'true',
 'as',
 'they',
 'did',
 'not',
 'have',
 'my',
 'phone',
 'number',
 'i',
 'was',
 'with',
 'a',
 'local',
 'guide',
 'and',
 'he',
 'had',
 'not',
 'been',
 'informed',
 'either',
 'i',
 'presume',
 'that',
 'i',
 'was',
 'bumped',
 'off',
 'the',
 'later',
 'flight',
 'did',
 'operate',
 'but',
 'as',
 'usual',
 'there',
 'was',

In [ ]:
#d) handling stopwords
x=set(stopwords.words('english'))
x.discard('not')
x.discard('very')


In [ ]:
#e) stemming
ps=PorterStemmer()
review=[ps.stem(word) for word in review if not word in x]

In [ ]:
#f) join the splitted data
review=' '.join(review)

In [ ]:
review

'not good airlin moroni anjouan veri small airlin ticket advis turn hr confus small airport direct offic ab aviat still close open hr told flight put back hr tri contact could not true not phone number local guid not inform either presum bump later flight oper usual confus check flight min problem not good airlin one comoro'

##NLP

In [ ]:
dataset

,Review_Title,Review,Recommended,final_review
0,pretty decent airline,Moroni to Moheli. Turned out to be a pretty ...,1,pretti decent airlin moroni mohe turn pretti d...
1,Not a good airline,Moroni to Anjouan. It is a very small airline...,0,not good airlin moroni anjouan veri small airl...
2,flight was fortunately short,Anjouan to Dzaoudzi. A very small airline an...,0,flight fortun short anjouan dzaoudzi veri smal...
4,it ruined our last days of holidays,Do not book a flight with this airline! My fr...,0,ruin last day holiday not book flight airlin f...
5,Had very bad experience,Had very bad experience with rerouted and ca...,0,veri bad experi veri bad experi rerout cancel ...
...,...,...,...,...
23164,$300 savings was absolutely not worth it,My issues started before we even flew. I rec...,0,save absolut not worth issu start even flew re...
23167,Avoid at all costs,Avoid at all costs. I booked flights to go f...,0,avoid cost avoid cost book flight go singapor ...
23168,Will not recommend to anyone,Flight was leaving at 23.15 and after an hou...,0,not recommend anyon flight leav hour half fli ...
23169,It was immaculately clean,Zipair is JAL’s budget airline. They don’t ha...,1,immacul clean zipair jal budget airlin class t...


In [ ]:
def preprocess(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    stop_words = set(stopwords.words('english'))
    stop_words.discard('not')
    stop_words.discard('very')
    text = [ps.stem(word) for word in text if word not in stop_words]
    return ' '.join(text)

In [ ]:
  dataset['final_review'] = dataset['final_review'].apply(preprocess)

<ipython-input-107-a20f79b1ce74>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['final_review'] = dataset['final_review'].apply(preprocess)


In [ ]:
dataset['final_review']

0        pretti decent airlin moroni mohe turn pretti d...
1        not good airlin moroni anjouan veri small airl...
2        flight fortun short anjouan dzaoudzi veri smal...
4        ruin last day holiday not book flight airlin f...
5        veri bad experi veri bad experi rerout cancel ...
                               ...                        
23164    save absolut not worth issu start even flew re...
23167    avoid cost avoid cost book flight go singapor ...
23168    not recommend anyon flight leav hour half fli ...
23169    immacul clean zipair jal budget airlin class t...
23170    lost money refund lie connect narita lost mone...
Name: final_review, Length: 12322, dtype: object

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
x = tfidf_vectorizer.fit_transform(dataset['final_review']).toarray()
y = dataset['Recommended']
y

0        1
1        0
2        0
4        0
5        0
        ..
23164    0
23167    0
23168    0
23169    1
23170    0
Name: Recommended, Length: 12322, dtype: int64

In [ ]:
#Split into train set and test set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0,stratify=y)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(9857, 2000)
(2465, 2000)
(9857,)
(2465,)


## ANN Model

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(units=128, activation="relu", input_shape=(x_train.shape[1],)),
    keras.layers.Dense(units=64, activation="relu"),
    keras.layers.Dense(units=1, activation="sigmoid")
])

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
309/309 [==============================] - 4s 5ms/step - loss: 0.2596 - accuracy: 0.8940
Epoch 2/10
309/309 [==============================] - 2s 5ms/step - loss: 0.1319 - accuracy: 0.9547
Epoch 3/10
309/309 [==============================] - 2s 5ms/step - loss: 0.1001 - accuracy: 0.9669
Epoch 4/10
309/309 [==============================] - 2s 5ms/step - loss: 0.0640 - accuracy: 0.9800
Epoch 5/10
309/309 [==============================] - 1s 5ms/step - loss: 0.0343 - accuracy: 0.9896
Epoch 6/10
309/309 [==============================] - 1s 5ms/step - loss: 0.0130 - accuracy: 0.9963
Epoch 7/10
309/309 [==============================] - 2s 8ms/step - loss: 0.0044 - accuracy: 0.9989
Epoch 8/10
309/309 [==============================] - 2s 6ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 9/10
309/309 [==============================] - 1s 5ms/step - loss: 8.1916e-04 - accuracy: 0.9999
Epoch 10/10
309/309 [==============================] - 1s 5ms/step - loss: 4.4367e-04 - accuracy

In [ ]:
model.save('review_checkers.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Evaluating Model

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

78/78 [==============================] - 0s 3ms/step - loss: 0.5047 - accuracy: 0.9266
Test loss: 0.5047131180763245
Test accuracy: 0.9265720248222351


In [ ]:
text = "will never come again"
text = preprocess(text)

In [ ]:
text

'never come'

In [ ]:
text_tfidf = tfidf_vectorizer.transform([text]).toarray()
y_predict = model.predict(text_tfidf)


1/1 [==============================] - 0s 113ms/step


In [ ]:
y_predict

array([[1.3996662e-05]], dtype=float32)

In [ ]:
if y_predict>0.5:
  print('positive')
else:
  print('negative')

negative
